In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import sys
import pickle
import random
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras as k
from matplotlib import pyplot as plt

#Load capsnet files
from capsnet import losses
from models import get_model

In [ ]:
# Cifar10 64x64 dataset could be downloaded from below given kaggle link
# https://www.kaggle.com/datasets/joaopauloschuler/cifar10-64x64-resized-via-cai-super-resolution

!unzip /content/gdrive/MyDrive/MSCOCO/cifar10-64x64.zip -d /content

In [ ]:
#https://raw.githubusercontent.com/MartinThoma/algorithms/master/ML/confusion-matrix/labels/cifar-100-labels.json

'''
CATEGORIES = ['class0','class1','class2','class3','class4','class5','class6','class7','class8','class9','class10',
              'class11','class12','class13','class14','class15','class16','class17','class18','class19','class20',
              'class21','class22','class23','class24','class25','class26','class27','class28','class29','class30',
              'class31','class32','class33','class34','class35','class36','class37','class38','class39','class40',
              'class41','class42','class43','class44','class45','class46','class47','class48','class49','class50',
              'class51','class52','class53','class54','class55','class56','class57','class58','class59','class60',
              'class61','class62','class63','class64','class65','class66','class67','class68','class69','class70',
              'class71','class72','class73','class74','class75','class76','class77','class78','class79','class80',
              'class81','class82','class83','class84','class85','class86','class87','class88','class89','class90',
              'class91','class92','class93','class94','class95','class96','class97','class98','class99']'''

#Cifar10 dataset has only 10 classes
CATEGORIES = ['class0','class1','class2','class3','class4','class5','class6','class7','class8','class9']              
SIZE = 64

# Directory for loading TRAIN/TEST raw images 
DATA_DIR = "/content/cifar10-64/test/"
SAVES_DIR = "/"


training_data = []

for category in CATEGORIES:
    category_path = os.path.join(DATA_DIR, category)
    for image_path in os.listdir(category_path):
        try:
            img_path = os.path.join(category_path, image_path)
            img_array = cv2.resize(cv2.imread(img_path, cv2.IMREAD_GRAYSCALE), (SIZE, SIZE))
            class_index = CATEGORIES.index(category)
            training_data.append([img_array, class_index])
        except:
            pass

random.shuffle(training_data)

images = []
labels = []

for image, label in training_data:
    images.append(image)
    labels.append(label)

In [ ]:
#Save train images
pickle_train_img = open("train_cifar10_64_images.pickle","wb")
pickle.dump(images, pickle_train_img)
pickle_train_img.close()

#Save train labels
pickle_train_label = open("train_cifar10_64_labels.pickle","wb")
pickle.dump(labels, pickle_train_label)
pickle_train_label.close()

In [ ]:
#Save test images
pickle_test_img = open("test_cifar10_64_images.pickle","wb")
pickle.dump(images, pickle_test_img)
pickle_test_img.close()

#Save test labels
pickle_test_label = open("test_cifar10_64_labels.pickle","wb")
pickle.dump(labels, pickle_test_label)
pickle_test_label.close()

In [ ]:
#Load train and test images, labels

x_train_pickle = open("/content/train_cifar10_64_images.pickle","rb")
x_train = pickle.load(x_train_pickle)

y_train_pickle = open("/content/train_cifar10_64_labels.pickle","rb")
y_train = pickle.load(y_train_pickle)

x_test_pickle = open("/content/test_cifar10_64_images.pickle","rb")
x_test = pickle.load(x_test_pickle)

y_test_pickle = open("/content/test_cifar10_64_labels.pickle","rb")
y_test = pickle.load(y_test_pickle)

In [ ]:
x_train = np.array(x_train).reshape(-1, 64, 64, 1).astype(np.float32)
x_test = np.array(x_test).reshape(-1, 64, 64, 1).astype(np.float32)

y_train = np.array(y_train).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

print("Total train images : {} : ".format(len(x_train)))
print("Total test images : {} : ".format(len(x_test)))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

Total train images : 50000 : 
Total test images : 10000 : 
(50000, 64, 64, 1)
(10000, 64, 64, 1)
(50000,)
(10000,)


In [ ]:
#Scale images
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
print(len(np.unique(y_train)))

10


In [ ]:
x_train.shape

(50000, 64, 64, 1)

In [ ]:
BASE_PATH = ""

ERR_FILE_NOT_FOUND = "file not found"


def main(x_train, x_test, y_train, y_test) :
    mode = "train"
    model_name = "original"
    dataset_name = "cifar100"

    NUM_CLASSES = len(np.unique(y_train))

    # configure model and print summary
    model = get_model(name=model_name, input_shape=x_train.shape[1:], num_classes=NUM_CLASSES)
    model.compile(optimizer=k.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0, clipvalue=0.5),
                loss=[lambda a, b: losses.margin_loss(a, b, 0.9, 0.01), 'mse'],
                loss_weights=[1, 5e-3],
                metrics={'pred': 'acc'})
    model.summary(line_length=150)

    filepath = f"{BASE_PATH}weights_{model_name}_{dataset_name}.hdf5"

    mode_name (model, mode, filepath)

def mode_name (model, mode, filepath):
    if mode == "retrain":
        # TODO

    if mode == "train" or mode == "retrain":
        checkpoint = k.callbacks.ModelCheckpoint(filepath, save_best_only=True)
        model.fit(x_train, [y_train, x_train],
              batch_size=8,
              epochs=1,
              validation_data=(x_test, (y_test, x_test)),
              callbacks=[checkpoint])

    if mode == "test":
        #TODO

    if mode == "demo":
        # TODO


if __name__ == '__main__':
    main(x_train, x_test, y_train, y_test)


Model: "original"
______________________________________________________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                     Param #           Connected to                                      
 input (InputLayer)                              [(None, 64, 64, 1)]              0                 []                                                
                                                                                                                                                      
 conv (Conv2D)                                   (None, 56, 56, 256)              20992             ['input[0][0]']                                   
                                                                                                                                                      
 conv_caps_2d (ConvCaps2D)                       (None, 24, 24, 32, 8)      

##TODO LIST

In [2]:
# Load Flickr dataset
# Preprocess dataset
# Extract feature using trained capsnet model weight
# Train in transformer to generate captions

# Calculate BELU score
# Evaluation the performance